In [7]:
import tensorflow as tf
import numpy as np

In [8]:
import json
import os
if os.path.exists('vocab_to_int.json'):
    vocab_to_int = json.load(open('vocab_to_int.json', 'r'))
    print('loaded vocab_to_int,  size: {}'.format(len(vocab_to_int)))
else:
    vocab_to_int = dict([(word, idx+1) for idx, word in enumerate(word_set)])
    json.dump(vocab_to_int, open('vocab_to_int.json', 'w'))

loaded vocab_to_int,  size: 74072


In [29]:
lstm_size = 256
lstm_layers = 2 #starting with lstm layer of 1 and if underfitting will expand to 2 
batch_size = 1
learning_rate = 0.001
embed_size = 300
seq_len = 200
n_words = len(vocab_to_int)

In [30]:
tf.reset_default_graph()
graph = tf.Graph()

with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, shape=(None, seq_len), name='inputs_')
    labels_ = tf.placeholder(tf.int32, shape=(None, 1), name='labels_') #this might be wrong
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    with tf.name_scope('embedding_layer'):

        embedding = tf.Variable(tf.random_uniform((n_words + 1, embed_size), -1.0, 1.0), name='embedding') 
        embed = tf.nn.embedding_lookup(embedding, inputs_)

    with tf.name_scope('lstm'):

        lstm = tf.contrib.rnn.BasicLSTMCell(num_units = lstm_size)

        #wraps lstm cell in a cell that applies dropout to the output of the lstm cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob) 
    
    with tf.name_scope('lstm_layer'):
        cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers) 
        initial_state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    with tf.name_scope('dynamic_rnn'):
        outputs, final_state = tf.nn.dynamic_rnn(cell = cell, inputs = embed, initial_state=initial_state)
   
    with tf.name_scope('prediction'):
        predictions = tf.contrib.layers.fully_connected(outputs[:, -1], num_outputs=1, activation_fn=tf.sigmoid) 

    saver = tf.train.Saver()
    merged = tf.summary.merge_all()

In [31]:
import json
import os
if os.path.exists('vocab_to_int.json'):
    vocab_to_int = json.load(open('vocab_to_int.json', 'r'))
    print('loaded vocab_to_int,  size: {}'.format(len(vocab_to_int)))
else:
    vocab_to_int = dict([(word, idx+1) for idx, word in enumerate(word_set)])
    json.dump(vocab_to_int, open('vocab_to_int.json', 'w'))

loaded vocab_to_int,  size: 74072


In [44]:
with open('./reviews.txt', 'r') as f:
    reviews = f.read()
    
reviews =reviews.split('\n')

In [80]:
from string import punctuation



def analyze_sentence(sent1):
    sent1 = ''.join([c for c in sent1.lower() if c not in punctuation])
    words = sent1.split()

    sub_sents = []

    for i in range(1, len(words) + 1):
        sub_sents.append(words[:i])
    print(sub_sents[-1])
    #skip words not in corpus
    sub_sents_ints = [[vocab_to_int[word] for word in sent if word in vocab_to_int] for sent in sub_sents]

    zero200_padding = lambda inp_list : [0 for _ in range(200 - len(inp_list))] + inp_list

    prior_out = 0.0
    with tf.Session(graph=graph) as sess:
        saver.restore(sess, './checkpoints/sentiment.ckpt-11')
        for i in range(min(seq_len, len(sub_sents_ints))):
            out = sess.run(predictions, feed_dict={inputs_:np.array([zero200_padding(sub_sents_ints[i])]),
                                         keep_prob: 1.0
        #                                 initial_state: init_state
                                        })
            #print(out[0][0])
            score = out[0, 0]
            print('{:.3f}; {:.2f}: \t {} \t {}'.format(score, score - prior_out, 'positive' if out > 0.5 else 'negative', words[i]))
            
            prior_out = score

In [81]:
sent1 = """
i watched this movie yesterday and thought it was greatest romantic comedy i have ever seen. 
In all of my days i have never enjoyed a movie as much as i've enjoyed this one and hope everyone will have a chance to see it.
"""
analyze_sentence(sent1)

['i', 'watched', 'this', 'movie', 'yesterday', 'and', 'thought', 'it', 'was', 'greatest', 'romantic', 'comedy', 'i', 'have', 'ever', 'seen', 'in', 'all', 'of', 'my', 'days', 'i', 'have', 'never', 'enjoyed', 'a', 'movie', 'as', 'much', 'as', 'ive', 'enjoyed', 'this', 'one', 'and', 'hope', 'everyone', 'will', 'have', 'a', 'chance', 'to', 'see', 'it']
0.198; 0.20: 	 negative 	 i
0.176; -0.02: 	 negative 	 watched
0.125; -0.05: 	 negative 	 this
0.111; -0.01: 	 negative 	 movie
0.096; -0.01: 	 negative 	 yesterday
0.096; -0.00: 	 negative 	 and
0.120; 0.02: 	 negative 	 thought
0.154; 0.03: 	 negative 	 it
0.143; -0.01: 	 negative 	 was
0.204; 0.06: 	 negative 	 greatest
0.339; 0.13: 	 negative 	 romantic
0.442; 0.10: 	 negative 	 comedy
0.533; 0.09: 	 positive 	 i
0.584; 0.05: 	 positive 	 have
0.559; -0.02: 	 positive 	 ever
0.677; 0.12: 	 positive 	 seen
0.675; -0.00: 	 positive 	 in
0.667; -0.01: 	 positive 	 all
0.630; -0.04: 	 positive 	 of
0.617; -0.01: 	 positive 	 my
0.691; 0.07: 

In [82]:
sent1 = """
I hate this movie. It has steven segal in it, who is terrible and most of all action movies should only have keanu reeves.
I'd rather watch two rats mating than have to see this again.
"""
analyze_sentence(sent1)

['i', 'hate', 'this', 'movie', 'it', 'has', 'steven', 'segal', 'in', 'it', 'who', 'is', 'terrible', 'and', 'most', 'of', 'all', 'action', 'movies', 'should', 'only', 'have', 'keanu', 'reeves', 'id', 'rather', 'watch', 'two', 'rats', 'mating', 'than', 'have', 'to', 'see', 'this', 'again']
0.198; 0.20: 	 negative 	 i
0.180; -0.02: 	 negative 	 hate
0.140; -0.04: 	 negative 	 this
0.129; -0.01: 	 negative 	 movie
0.142; 0.01: 	 negative 	 it
0.145; 0.00: 	 negative 	 has
0.134; -0.01: 	 negative 	 steven
0.118; -0.02: 	 negative 	 segal
0.121; 0.00: 	 negative 	 in
0.144; 0.02: 	 negative 	 it
0.140; -0.00: 	 negative 	 who
0.134; -0.01: 	 negative 	 is
0.078; -0.06: 	 negative 	 terrible
0.087; 0.01: 	 negative 	 and
0.105; 0.02: 	 negative 	 most
0.122; 0.02: 	 negative 	 of
0.122; -0.00: 	 negative 	 all
0.143; 0.02: 	 negative 	 action
0.212; 0.07: 	 negative 	 movies
0.254; 0.04: 	 negative 	 should
0.188; -0.07: 	 negative 	 only
0.153; -0.04: 	 negative 	 have
0.137; -0.02: 	 negat

In [83]:
analyze_sentence('this is the single greatest film of all time i cant wait until they make a sequel')

['this', 'is', 'the', 'single', 'greatest', 'film', 'of', 'all', 'time', 'i', 'cant', 'wait', 'until', 'they', 'make', 'a', 'sequel']
0.176; 0.18: 	 negative 	 this
0.155; -0.02: 	 negative 	 is
0.157; 0.00: 	 negative 	 the
0.167; 0.01: 	 negative 	 single
0.227; 0.06: 	 negative 	 greatest
0.292; 0.07: 	 negative 	 film
0.353; 0.06: 	 negative 	 of
0.366; 0.01: 	 negative 	 all
0.342; -0.02: 	 negative 	 time
0.398; 0.06: 	 negative 	 i
0.388; -0.01: 	 negative 	 cant
0.536; 0.15: 	 positive 	 wait
0.566; 0.03: 	 positive 	 until
0.635; 0.07: 	 positive 	 they
0.572; -0.06: 	 positive 	 make
0.593; 0.02: 	 positive 	 a
0.616; 0.02: 	 positive 	 sequel


In [85]:
# a some what negative review of one flew over the cuckoos nest
#http://www.rogerebert.com/reviews/one-flew-over-the-cuckoos-nest-1975

sent = """
Another scene that just doesn't work, because it's too heavily burdened with its purpose, occurs when McMurphy escapes, commandeers a school bus, and takes all the inmates of the ward on a fishing trip in a stolen boat. The scene causes an almost embarrassing break in the movie -- it's Forman's first serious misstep -- because it's an idealized fantasy in the midst of realism. By now, we've met the characters, we know them in the context of hospital politics, and when they're set down on the boat deck, they just don't belong there. The ward is the arena in which they'll win or lose, and it's not playing fair -- to them, as characters -- to give them a fishing trip.
Even as I'm making these observations, though, I can't get out of my mind the tumultuous response that "Cuckoo's Nest" received from its original audiences. Even the most obvious, necessary, and sobering scenes -- as when McMurphy tries to strangle Nurse Ratched to death -- were received, not seriously, but with sophomoric cheers and applause. Maybe that's the way to get the most out of the movie -- see it as a simple-minded antiestablishment parable -- but I hope not. I think there are long stretches of a very good film to be found in the midst of Forman's ultimate failure, and I hope they don't get drowned in the applause for the bad stuff that plays to the galleries.

"""

analyze_sentence(sent)

['another', 'scene', 'that', 'just', 'doesnt', 'work', 'because', 'its', 'too', 'heavily', 'burdened', 'with', 'its', 'purpose', 'occurs', 'when', 'mcmurphy', 'escapes', 'commandeers', 'a', 'school', 'bus', 'and', 'takes', 'all', 'the', 'inmates', 'of', 'the', 'ward', 'on', 'a', 'fishing', 'trip', 'in', 'a', 'stolen', 'boat', 'the', 'scene', 'causes', 'an', 'almost', 'embarrassing', 'break', 'in', 'the', 'movie', 'its', 'formans', 'first', 'serious', 'misstep', 'because', 'its', 'an', 'idealized', 'fantasy', 'in', 'the', 'midst', 'of', 'realism', 'by', 'now', 'weve', 'met', 'the', 'characters', 'we', 'know', 'them', 'in', 'the', 'context', 'of', 'hospital', 'politics', 'and', 'when', 'theyre', 'set', 'down', 'on', 'the', 'boat', 'deck', 'they', 'just', 'dont', 'belong', 'there', 'the', 'ward', 'is', 'the', 'arena', 'in', 'which', 'theyll', 'win', 'or', 'lose', 'and', 'its', 'not', 'playing', 'fair', 'to', 'them', 'as', 'characters', 'to', 'give', 'them', 'a', 'fishing', 'trip', 'even',